In [129]:
import pandas as pd
import numpy as np
import tensorflow as tf
import math
import heapq
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

In [130]:
# Đọc data từ file CSV
origin_data = pd.read_csv('./customer_product_dtmtb.csv')
customer_product_df = origin_data[['customerId', 'productId', 'rating']]

In [131]:
customer_product_df

,customerId,productId,rating
0,245967,271046571,5
1,20220461,271046571,5
2,17677024,271046571,5
3,1316594,271046571,5
4,11567233,271046571,5
...,...,...,...
607,6313819,191089547,5
608,22202828,191089547,5
609,19013048,205750556,5
610,17556056,123345348,5


Chuyển rating về giá trị từ 0 đến 1

In [132]:
# Khởi tạo MinMaxScaler với range từ 0 đến 1
scaler = MinMaxScaler(feature_range=(0, 1))

# Chọn cột cần chuyển đổi
column_to_scale = 'rating'

# Chuyển đổi cột thành giá trị 0-1
scaled_column = scaler.fit_transform(customer_product_df[[column_to_scale]])

# Ghi đè lên cột ban đầu trong DataFrame
customer_product_df.loc[:, column_to_scale] = scaled_column.flatten()

# In ra DataFrame đã được chuyển đổi
print(customer_product_df)


     customerId  productId  rating
0        245967  271046571     1.0
1      20220461  271046571     1.0
2      17677024  271046571     1.0
3       1316594  271046571     1.0
4      11567233  271046571     1.0
..          ...        ...     ...
607     6313819  191089547     1.0
608    22202828  191089547     1.0
609    19013048  205750556     1.0
610    17556056  123345348     1.0
611     7068044  214756058     1.0

[612 rows x 3 columns]


## Chuyển đổi dataframe về dạng matrix

In [133]:
# Tạo DataFrame mới
product_ids = customer_product_df['productId'].unique()
customer_ids = customer_product_df['customerId'].unique()

# Tạo DataFrame mới với tất cả giá trị ban đầu là 0
df = pd.DataFrame(0.0, index=customer_ids, columns=product_ids)

# Lặp qua từng dòng trong customer_product_df
for index, row in customer_product_df.iterrows():
    customer_id = row['customerId']
    product_id = row['productId']
    rating = row['rating']
    
    # Thay thế số 0 bằng giá trị cột 'rating' tương ứng, đã chuyển đổi sang float
    df.at[customer_id, product_id] = float(rating)

# Hiển thị DataFrame mới
print(df.head())


          271046571  184036446  184059211  183330021  57809866   43924153   \
245967          1.0        0.0        0.0        0.0        0.0        0.0   
20220461        1.0        0.0        0.0        0.0        0.0        0.0   
17677024        1.0        0.0        0.0        0.0        0.0        0.0   
1316594         1.0        0.0        0.0        0.0        0.0        0.0   
11567233        1.0        0.0        0.0        0.0        1.0        0.0   

          120295859  123345348  244308977  6597807    ...  274095168  \
245967          0.0        0.0        0.0        0.0  ...        0.0   
20220461        0.0        0.0        0.0        0.0  ...        0.0   
17677024        0.0        0.0        0.0        0.0  ...        0.0   
1316594         0.0        0.0        0.0        0.0  ...        0.0   
11567233        0.0        0.0        0.0        0.0  ...        0.0   

          271972435  270447782  197670818  247829297  205750556  273175604  \
245967          1.0 

# Collaborative Filtering - Cosine

In [134]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr


### Tạo ma trận rating 

#### Hàm tính ma trận similarity - Pearson corralation coeficient

In [135]:
# Hàm tính Pearson correlation coefficient giữa hai vector
def pearson_similarity(x, y):
    mask = np.logical_and(~np.isnan(x), ~np.isnan(y))
    if np.sum(mask) > 1:
        return pearsonr(x[mask], y[mask])[0]
    else:
        return 0  # Trường hợp không đủ dữ liệu để tính toán

# Hàm tính ma trận similarity giữa các items dựa trên Pearson correlation coefficient
def item_similarity_matrix_pearson(matrix):
    num_items = matrix.shape[1]
    similarity_matrix = np.zeros((num_items, num_items))
    for i in range(num_items):
        for j in range(num_items):
            similarity_matrix[i, j] = pearson_similarity(matrix[:, i], matrix[:, j])
    return similarity_matrix

#### Hàm tính ma trận similarity - Adjusted Cosine Similarity

In [136]:
# Hàm tính Adjusted Cosine Similarity giữa hai vector
def adjusted_cosine_similarity(x, y):
    mask = np.logical_and(~np.isnan(x), ~np.isnan(y))
    if np.sum(mask) > 1:
        x_mean = np.mean(x[mask])
        y_mean = np.mean(y[mask])
        numerator = np.sum((x[mask] - x_mean) * (y[mask] - y_mean))
        denominator = np.sqrt(np.sum((x[mask] - x_mean) ** 2) * np.sum((y[mask] - y_mean) ** 2))
        if denominator != 0:
            return numerator / denominator
        else:
            return 0  # Trường hợp mẫu số bằng 0
    else:
        return 0  # Trường hợp không đủ dữ liệu để tính toán

# Hàm tính ma trận similarity giữa các items dựa trên Adjusted Cosine Similarity
def item_similarity_matrix_adjusted_cosine(matrix):
    num_items = matrix.shape[1]
    similarity_matrix = np.zeros((num_items, num_items))
    for i in range(num_items):
        for j in range(num_items):
            similarity_matrix[i, j] = adjusted_cosine_similarity(matrix[:, i], matrix[:, j])
    return similarity_matrix

#### Hàm dự đoán rating

In [137]:
def predict_ratings(similarity_matrix, matrix):
    """Hàm dự đoán rating cho các cặp người dùng-item chưa được rating dựa trên similarity matrix"""
    num_users, num_items = matrix.shape
    predicted_ratings = np.zeros((num_users, num_items))
    for i in range(num_users):
        for j in range(num_items):
            if np.isnan(matrix[i, j]):  # Chỉ dự đoán cho các cặp người dùng-item chưa được rating
                rated_indices = np.where(~np.isnan(matrix[i]))[0]  # Các mục đã được rating
                rated_similarities = similarity_matrix[j, rated_indices]  # Tương đồng giữa mục cần dự đoán và các mục đã được rating
                rated_ratings = matrix[i, rated_indices]  # Các ratings của các mục đã được rating
                predicted_ratings[i, j] = np.dot(rated_similarities, rated_ratings) / np.sum(np.abs(rated_similarities))
    return predicted_ratings

#### Tính ma trận

In [138]:
# Tính ma trận similarity giữa các items dựa trên Pearson correlation coefficient
similarity_matrix_pearson = item_similarity_matrix_pearson(df.values)

# Dự đoán rating cho các cặp người dùng-item chưa được rating dựa trên Pearson correlation coefficient
predicted_ratings_pearson = predict_ratings(similarity_matrix_pearson, df.values)

# In ra ma trận dự đoán
print("Predicted Ratings (Pearson):")
print(predicted_ratings_pearson)

Predicted Ratings (Pearson):
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [139]:
# Tính ma trận similarity giữa các items dựa trên Adjusted Cosine Similarity
similarity_matrix_adjusted_cosine = item_similarity_matrix_adjusted_cosine(df.values)

# Dự đoán rating cho các cặp người dùng-item chưa được rating dựa trên Adjusted Cosine Similarity
predicted_ratings_adjusted_cosine = predict_ratings(similarity_matrix_adjusted_cosine, df.values)

# In ra ma trận dự đoán
print("Predicted Ratings (Adjusted Cosine):")
print(predicted_ratings_adjusted_cosine)

Predicted Ratings (Adjusted Cosine):
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


### Dự đoán product cho user 0

In [140]:
from sklearn.metrics import mean_squared_error

# User ID cần dự đoán
user_id = 0

# Rating thực tế của user_id
true_ratings = df.values[user_id]

# Dự đoán rating cho user_id sử dụng Pearson correlation coefficient
predicted_ratings_pearson_user = predicted_ratings_pearson[user_id]
top_5_recommendations = predicted_ratings_pearson_user.argsort()[-5:][::-1]
print("Pearson correlation coefficient - Recommended Items for User 0:\n", top_5_recommendations)

# Dự đoán rating cho user_id sử dụng Adjusted Cosine Similarity
predicted_ratings_adjusted_cosine_user = predicted_ratings_adjusted_cosine[user_id]
top_5_recommendations = predicted_ratings_adjusted_cosine_user.argsort()[-5:][::-1]
print("Adjusted Cosine Similarity - Recommended Items for User 0:\n", top_5_recommendations)

Pearson correlation coefficient - Recommended Items for User 0:
 [74 18 20 21 22]
Adjusted Cosine Similarity - Recommended Items for User 0:
 [74 18 20 21 22]


### Đánh giá mô hình

In [141]:
# Tính MSE cho Pearson correlation coefficient
mse_pearson = mean_squared_error(true_ratings, predicted_ratings_pearson_user)

# Tính RMSE cho Pearson correlation coefficient
rmse_pearson = np.sqrt(mse_pearson)

# Tính MSE cho Adjusted Cosine Similarity
mse_adjusted_cosine = mean_squared_error(true_ratings, predicted_ratings_adjusted_cosine_user)

# Tính RMSE cho Adjusted Cosine Similarity
rmse_adjusted_cosine = np.sqrt(mse_adjusted_cosine)

print("MSE (Pearson):", mse_pearson)
print("RMSE (Pearson):", rmse_pearson)

print("MSE (Adjusted Cosine):", mse_adjusted_cosine)
print("RMSE (Adjusted Cosine):", rmse_adjusted_cosine)

MSE (Pearson): 0.04592592592592592
RMSE (Pearson): 0.21430335024428787
MSE (Adjusted Cosine): 0.04592592592592592
RMSE (Adjusted Cosine): 0.21430335024428787


# Collaborative Filtering - KNN

In [142]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Chuyển đổi tên cột thành chuỗi
df.columns = df.columns.astype(str)

# Xây dựng mô hình KNN
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(df)

NearestNeighbors(algorithm='brute', metric='cosine')

### Đề xuất cho Customer 0

In [143]:
# Dự đoán rating cho userId thứ 0
user_id = 0
user_ratings = df.iloc[user_id]  # Lấy các rating của user
new_data = user_ratings.values.reshape(1, -1)  # Chuyển đổi thành mảng 2D
distances, indices = knn_model.kneighbors(new_data)  # Tìm các user láng giềng gần nhất
neighbor_ratings = df.iloc[indices[0]]  # Lấy các rating của các user láng giềng
predicted_ratings = neighbor_ratings.mean(axis=0)  # Dự đoán rating bằng trung bình của các user láng giềng

# In ra rating dự đoán cho userId thứ 0
print("Predicted ratings for userId 0:", predicted_ratings)

Predicted ratings for userId 0: 271046571    0.6
184036446    0.0
184059211    0.0
183330021    0.0
57809866     0.0
            ... 
205750556    0.0
273175604    0.0
274101255    0.0
214756058    0.0
274037360    0.0
Length: 75, dtype: float64


C:\Users\HUYNH THIEN NHAN\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [144]:
# Lấy top 5 đề xuất từ các predicted ratings
top_5_recommendations = predicted_ratings.argsort()[-5:][::-1]

print("Top 5 Recommended Items for User 0:\n", top_5_recommendations)

Top 5 Recommended Items for User 0:
 274037360     0
214756058    66
274101255    19
273175604    14
205750556    33
dtype: int64


### Đánh giá mô hình

In [145]:
# Đánh giá mô hình bằng mean squared error
actual_ratings = df.iloc[user_id]  # Lấy các rating thực tế của userId 0
mse = mean_squared_error(actual_ratings, predicted_ratings)
print("Mean squared error:", mse)

Mean squared error: 0.015525925925925922
